# Car price prediction


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('../input/vehicle-dataset-from-cardekho/car data.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
# checking unique values of the categorical features
print(df['Seller_Type'].unique())
print(df['Transmission'].unique())
print(df['Fuel_Type'].unique())
print(df['Owner'].unique())

In [ ]:
# checking null values or missing data
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
df['current year'] = 2020

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
final_dataset = df[['Year', 'Selling_Price', 'Present_Price', 'Kms_Driven',
       'Fuel_Type', 'Seller_Type', 'Transmission', 'Owner', 'current year']]

In [ ]:
final_dataset.head()

In [ ]:
final_dataset['no of year'] = final_dataset['current year'] - final_dataset['Year']

In [ ]:
final_dataset.head()

In [ ]:
final_dataset.drop(['Year'], axis=1, inplace=True)

In [ ]:
final_dataset.drop(['current year'], axis=1, inplace=True)

In [ ]:
final_dataset.head()

In [ ]:
final_dataset=pd.get_dummies(final_dataset, drop_first=True)

In [ ]:
final_dataset.head()

In [ ]:
final_dataset.corr()

In [ ]:
import seaborn as sns

In [ ]:
sns.pairplot(final_dataset)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
corrmat = final_dataset.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(20,20))
# plot heatmap
g = sns.heatmap(final_dataset[top_corr_features].corr(), annot=True, cmap = 'RdYlGn')

In [ ]:
final_dataset.head()

In [ ]:
# independent feature
X = final_dataset.iloc[:, 1:]

# dependent feature
y = final_dataset.iloc[:, 0]

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
## Feature Importance
from sklearn.ensemble import ExtraTreesRegressor
model = ExtraTreesRegressor()
model.fit(X,y)

In [ ]:
print(model.feature_importances_)

In [ ]:
# plotting the feature importances
feature_importance = pd.Series(model.feature_importances_, index=X.columns)
feature_importance.plot(kind='barh')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
X_train.shape


In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_random = RandomForestRegressor()

In [ ]:
import numpy as np

In [ ]:
## Hyperparameters
n_estimators = [int(x) for x in np.linspace(start=100 ,stop=1200, num=12)]
print(n_estimators)

In [ ]:
# Randomized search CV

# no of trees in random forest 
n_estimators = [int(x) for x in np.linspace(start=100 ,stop=1200, num=12)]

# No of features to consider at every split 
max_features = ['auto', 'sqrt']

# Maximum no of levels in a tree
max_depth = [int(x) for x in np.linspace(5, 30, num=6)]

# minimum number of samples required to split a node 
min_samples_split = [2, 5, 10, 15, 100]

# minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# create a random grid 

random_grid = { 'n_estimators':n_estimators,
                'max_features': max_features,
                'max_depth': max_depth,
                'min_samples_split': min_samples_split,
                'min_samples_leaf': min_samples_leaf}
print(random_grid)

In [ ]:
# Use the random grid to search for the best hyer parameter 
# First create the base model to tune
rf = RandomForestRegressor()

In [ ]:
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid, scoring='neg_mean_squared_error', n_iter=10, cv=5, verbose=2, random_state=42, n_jobs=1)

### Fitting the model

In [ ]:
rf_random.fit(X_train, y_train)

In [ ]:
predictions = rf_random.predict(X_test)

In [ ]:
predictions

In [ ]:
sns.distplot(y_test-predictions)

In [ ]:
plt.scatter(y_test, predictions)

In [ ]:
import pickle

# open a file where you want to store the data
file = open('random_forest_regression_model.pkl', 'wb')

# dump info to that file
pickle.dump(rf_random, file)